In [1]:
import pandas as pd 
import psycopg2
import getpass
from helpers import *

connection = psycopg2.connect(
    user = 'postgres',
    database ="mimic",
    password=getpass.getpass("Enter postgres password"), 
    host="127.0.0.1", 
    port = "5433",
    options=f'-c search_path=mimiciii')

##############################################################################
####################### CONNECT TO MIMIC #####################################

# Create our test query
test_query = """
SELECT subject_id, hadm_id, admittime, dischtime, admission_type, diagnosis
FROM admissions
"""

# Run the query and assign the results to a variable
test = pd.read_sql_query(test_query,connection)

print("TEST:", test.head())

if isinstance(test, pd.DataFrame):
    print("Good to go!")
else:
    print("This is not a data  frame! something is wrong, check before you continue with the queries") 

TEST:    subject_id  hadm_id           admittime           dischtime admission_type  \
0          22   165315 2196-04-09 12:26:00 2196-04-10 15:54:00      EMERGENCY   
1          23   152223 2153-09-03 07:15:00 2153-09-08 19:10:00       ELECTIVE   
2          23   124321 2157-10-18 19:34:00 2157-10-25 14:00:00      EMERGENCY   
3          24   161859 2139-06-06 16:14:00 2139-06-09 12:48:00      EMERGENCY   
4          25   129635 2160-11-02 02:06:00 2160-11-05 14:55:00      EMERGENCY   

                                           diagnosis  
0                            BENZODIAZEPINE OVERDOSE  
1  CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...  
2                                         BRAIN MASS  
3                     INTERIOR MYOCARDIAL INFARCTION  
4                            ACUTE CORONARY SYNDROME  
Good to go!


## Define the mortality cohort

In [2]:
######################################################################################
######################################################################################

#  COHORT SELECTION

######################################################################################
######################## query admissions, patients ##################################


# define theta_age, theta_history, theta_length_of_stay

# For demographic information, patient’s age, gender, marital status, ethnicity, and insurance information are considered. 
# Only adult patients are enrolled in this study. 
# Hence, age was split into 5 groups (18, 25), (25, 45), (45, 65), (65, 89), (89,). 
# For admission-related information, admission type is included as features.
# Choose ICU patients admitted for the first time in ICU, exclude later ICU admissions
# define desired duration of stay, theta_icu_stay_min and theta_icu_stay_max in hours
#  
# theta_age = input("Define the age limit for the cohort. For adults choose 18")
# theta_history = input("Define how far back to look in the patients history past. Input number in years")
# theta_length_of_stay = input("Define hospital admissions whose length of stay is less than x day to be excluded")

icu_demographics = pd.read_sql("""

SELECT 
icustays.subject_id, 
icustays.hadm_id, 
icustays.icustay_id, 
patients.DOB, 
patients.DOD, 
patients.gender, 
admissions.ethnicity, 
admissions.diagnosis as diagnosis_at_admission, 
icustays.intime 
, cast(icustays.outtime as date) - cast(icustays.intime as date)  as icu_length_of_stay
, (cast(icustays.INTIME as date) - cast(patients.DOB as date))/ 364.242 as age
, (cast(admissions.deathtime as date) - cast(icustays.intime as date)) as death_after_icu_hours
, admissions.hospital_expire_flag , icustays.outtime 
, RANK() OVER (PARTITION BY icustays.subject_id ORDER BY icustays.intime) AS icustay_id_order 
, CASE when admissions.deathtime between icustays.intime and icustays.outtime THEN 1 ELSE 0 END AS mort_icu
, CASE when admissions.deathtime between admissions.admittime and admissions.dischtime THEN 1 ELSE 0 END AS mort_hosp


FROM icustays
INNER JOIN patients 
  ON icustays.subject_id = patients.subject_id
INNER JOIN admissions
    ON admissions.subject_id = icustays.subject_id
    AND admissions.hadm_id = icustays.hadm_id

""", con= connection)


In [3]:
icu_demographics

,subject_id,hadm_id,icustay_id,dob,dod,gender,ethnicity,diagnosis_at_admission,intime,icu_length_of_stay,age,death_after_icu_hours,hospital_expire_flag,outtime,icustay_id_order,mort_icu,mort_hosp
0,2,163353,243653,2138-07-17,NaT,M,ASIAN,NEWBORN,2138-07-17 21:20:07,0.0,0.000000,NaN,0,2138-07-17 23:32:21,1,0,0
1,3,145834,211552,2025-04-11,2102-06-14,M,WHITE,HYPOTENSION,2101-10-20 19:10:11,6.0,76.734698,NaN,0,2101-10-26 20:43:09,1,0,0
2,4,185777,294638,2143-05-12,NaT,F,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-16 00:29:31,1.0,47.976345,NaN,0,2191-03-17 16:46:31,1,0,0
3,5,178980,214757,2103-02-02,NaT,M,ASIAN,NEWBORN,2103-02-02 06:04:24,0.0,0.000000,NaN,0,2103-02-02 08:06:00,1,0,0
4,6,107064,228232,2109-06-21,NaT,F,WHITE,CHRONIC RENAL FAILURE/SDA,2175-05-30 21:30:54,4.0,66.120876,NaN,0,2175-06-03 13:39:54,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61527,99985,176670,279638,2127-04-08,NaT,M,WHITE,FEVER,2181-01-29 05:33:34,11.0,53.961377,NaN,0,2181-02-09 12:45:20,1,0,0
61528,99991,151118,226241,2137-04-07,NaT,M,WHITE,DIVERTICULITIS/SDA,2184-12-28 17:30:58,3.0,47.858292,NaN,0,2184-12-31 20:56:20,1,0,0
61529,99992,197084,242052,2078-10-17,NaT,F,WHITE,RETROPERITONEAL HEMORRHAGE,2144-07-25 18:04:42,2.0,65.950659,NaN,0,2144-07-27 17:27:55,1,0,0
61530,99995,137810,229633,2058-05-29,2147-09-29,F,WHITE,ABDOMINAL AORTIC ANEURYSM/SDA,2147-02-08 13:53:58,2.0,88.940869,NaN,0,2147-02-10 17:46:30,1,0,0


In [4]:
print("Number of patients hospitalized: ", len(set((icu_demographics.subject_id))))


Number of patients hospitalized:  46476


In [5]:
icu_demographics.head()

,subject_id,hadm_id,icustay_id,dob,dod,gender,ethnicity,diagnosis_at_admission,intime,icu_length_of_stay,age,death_after_icu_hours,hospital_expire_flag,outtime,icustay_id_order,mort_icu,mort_hosp
0,2,163353,243653,2138-07-17,NaT,M,ASIAN,NEWBORN,2138-07-17 21:20:07,0.0,0.000000,NaN,0,2138-07-17 23:32:21,1,0,0
1,3,145834,211552,2025-04-11,2102-06-14,M,WHITE,HYPOTENSION,2101-10-20 19:10:11,6.0,76.734698,NaN,0,2101-10-26 20:43:09,1,0,0
2,4,185777,294638,2143-05-12,NaT,F,WHITE,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-16 00:29:31,1.0,47.976345,NaN,0,2191-03-17 16:46:31,1,0,0
3,5,178980,214757,2103-02-02,NaT,M,ASIAN,NEWBORN,2103-02-02 06:04:24,0.0,0.000000,NaN,0,2103-02-02 08:06:00,1,0,0
4,6,107064,228232,2109-06-21,NaT,F,WHITE,CHRONIC RENAL FAILURE/SDA,2175-05-30 21:30:54,4.0,66.120876,NaN,0,2175-06-03 13:39:54,1,0,0


In [6]:
# include patients only on the 1st icu stay, older than 18 and with a length of stay from 1 to 10 days 

icu_demographics_filter = icu_demographics[(icu_demographics.icustay_id_order == 1) & (icu_demographics.age >= 18 )]
print("Number of patients hospitalized: ", len(set((icu_demographics_filter.subject_id))))

icu_demographics_filter = icu_demographics_filter[(icu_demographics_filter.icu_length_of_stay >= 0.5) & (icu_demographics_filter.icu_length_of_stay <= 10)]
print("Number of patients hospitalized: ", len(set((icu_demographics_filter.subject_id))))


Number of patients hospitalized:  38512
Number of patients hospitalized:  34651


In [7]:
icu_demographics_filter = icu_demographics_filter.drop(["icustay_id_order"], axis=1)

In [8]:
print("Number of patients with in hospital mortality: ", icu_demographics_filter.mort_hosp.value_counts())
print("Number of patients with in icu mortality: ", icu_demographics_filter.mort_icu.value_counts())

Number of patients with in hospital mortality:  0    31309
1     3342
Name: mort_hosp, dtype: int64
Number of patients with in icu mortality:  0    32354
1     2297
Name: mort_icu, dtype: int64


In [9]:
icu_demographics_filter.to_hdf('data_icu.h5', key='demographics', mode='w') 

c:\Users\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\.venv\lib\site-packages\pandas\core\generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['gender', 'ethnicity', 'diagnosis_at_admission'], dtype='object')]

  encoding=encoding,


icu_demographics_filter.hadm_id.value_counts().sort_values()

In [10]:
subject_id_set = set(icu_demographics_filter['subject_id'])
hadm_id_set = set(icu_demographics_filter['hadm_id'])

## Labs and vitals 

In [11]:
var_map = get_variable_mapping("resources/itemid_to_variable_map.csv")
var_map

,LEVEL2,LEVEL1,LINKSTO
itemid,,,
50861,Alanine aminotransferase,Alanine aminotransferase,labevents
769,Alanine aminotransferase,Alanine aminotransferase,chartevents
220644,Alanine aminotransferase,Alanine aminotransferase,chartevents
50862,Albumin,Albumin,labevents
772,Albumin,Albumin,chartevents
...,...,...,...
40473,Urine output,Urine output,outputevents
40715,Urine output,Urine output,outputevents
43175,Urine output,Urine output,outputevents


In [12]:
chartitems_to_keep = set(var_map.loc[var_map['LINKSTO'] == 'chartevents'].index)
#chartitems_to_keep = set([ str(i) for i in chartitems_to_keep ])

labitems_to_keep = set(var_map.loc[var_map['LINKSTO'] == 'labevents'].index)
#labitems_to_keep = set([ str(i) for i in labitems_to_keep ])


subject_id_set = set(icu_demographics_filter['subject_id'])
hadm_id_set = set(icu_demographics_filter['hadm_id'])
icustay_id_set = set(icu_demographics_filter['icustay_id'])

In [13]:
query = \
"""
select chartevents.subject_id, icustays.hadm_id, chartevents.icustay_id, chartevents.charttime, chartevents.itemid, chartevents.value, valueuom
FROM icustays 
INNER JOIN chartevents  ON icustays.icustay_id = chartevents.icustay_id
where chartevents.icustay_id in %(icu_ids)s
and chartevents.itemid in %(charts_to_keep)s
and chartevents.charttime between intime and outtime -- in between the icu stay
and chartevents.error is distinct from 1 -- filter wrong charts
and chartevents.valuenum is not null
UNION ALL
select distinct icustays.subject_id, icustays.hadm_id, icustays.icustay_id, labevents.charttime, labevents.itemid, labevents.value, valueuom
FROM icustays 
INNER JOIN labevents ON icustays.hadm_id = labevents.hadm_id
where icustays.icustay_id in %(icu_ids)s
and labevents.itemid in %(labs_to_keep)s
and labevents.charttime between (intime - interval '6' hour) and outtime
and labevents.valuenum > 0 -- filter wrong lab values
;
"""
labs_vitals= pd.read_sql_query(query, connection, params={'icu_ids': tuple(icustay_id_set), 'charts_to_keep': tuple(chartitems_to_keep), 'labs_to_keep': tuple(labitems_to_keep)})


In [14]:
labs_vitals.head()

,subject_id,hadm_id,icustay_id,charttime,itemid,value,valueuom
0,4,185777,294638,2191-03-16 08:00:00,646,100,%
1,26,197661,244882,2126-05-09 04:30:00,1522,8.8000001907348633,mg/dl
2,26,197661,244882,2126-05-09 04:30:00,1523,104,None
3,26,197661,244882,2126-05-09 04:30:00,1525,1.4,None
4,26,197661,244882,2126-05-09 04:30:00,1529,106,None


In [15]:
labs_vitals.to_hdf('data_icu.h5', key='labs_vitals') 

c:\Users\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\.venv\lib\site-packages\pandas\core\generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['value', 'valueuom'], dtype='object')]

  encoding=encoding,


## Vasopressors 

In [16]:
### from metavision
## from https://gitlab.doc.ic.ac.uk/AIClinician/AIClinician/-/blob/master/AIClinician_Data_extract_MIMIC3_140219.ipynb
query = """
select inputevents_mv.subject_id, inputevents_mv.hadm_id, inputevents_mv.icustay_id, inputevents_mv.itemid, inputevents_mv.starttime, inputevents_mv.endtime, inputevents_mv.amount, -- rate, -- ,rateuom,
case when itemid in (30120,221906,30047) and rateuom='mcg/kg/min' then round(cast(rate as numeric),3)  -- norad
when itemid in (30120,221906,30047) and rateuom='mcg/min' then round(cast(rate/80 as numeric),3)  -- norad
when itemid in (30119,221289) and rateuom='mcg/kg/min' then round(cast(rate as numeric),3) -- epi
when itemid in (30119,221289) and rateuom='mcg/min' then round(cast(rate/80 as numeric),3) -- epi
when itemid in (30051,222315) and rate > 0.2 then round(cast(rate*5/60  as numeric),3) -- vasopressin, in U/h
when itemid in (30051,222315) and rateuom='units/min' then round(cast(rate*5 as numeric),3) -- vasopressin
when itemid in (30051,222315) and rateuom='units/hour' then round(cast(rate*5/60 as numeric),3) -- vasopressin
when itemid in (30128,221749,30127) and rateuom='mcg/kg/min' then round(cast(rate*0.45 as numeric),3) -- phenyl
when itemid in (30128,221749,30127) and rateuom='mcg/min' then round(cast(rate*0.45 / 80 as numeric),3) -- phenyl
when itemid in (221662,30043,30307) and rateuom='mcg/kg/min' then round(cast(rate*0.01 as numeric),3)  -- dopa
when itemid in (221662,30043,30307) and rateuom='mcg/min' then round(cast(rate*0.01/80 as numeric),3) else null end as rate_std-- dopa
, round(case when inputevents_mv.amountuom = 'L' then inputevents_mv.amount * 1000.0 when inputevents_mv.amountuom = 'ml' then inputevents_mv.amount else null end) as amount
, case when inputevents_mv.subject_id in %(subject_ids)s then 1 else 0 end as has_vasopressors-- binary indicator of whether patients received vassos
from inputevents_mv
INNER JOIN icustays  ON icustays.icustay_id = inputevents_mv.icustay_id
where itemid in (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307) and rate is not null and statusdescription <> 'Rewritten'
and inputevents_mv.icustay_id in %(icu_ids)s

--and inputevents_mv.starttime between icustays.intime and icustays.outtime -- in between the icu stay
order by icustay_id, itemid, starttime

"""

vaso_mv = pd.read_sql_query(query,connection, params={'subject_ids': tuple(subject_id_set), 'icu_ids': tuple(icustay_id_set)})

In [17]:
vaso_mv

,subject_id,hadm_id,icustay_id,itemid,starttime,endtime,amount,rate_std,amount,has_vasopressors
0,41710,181955,200028,221906,2133-10-29 17:49:00,2133-10-29 18:11:00,0.110901,0.060,None,1
1,41710,181955,200028,221906,2133-10-29 18:11:00,2133-10-29 18:19:00,0.053759,0.080,None,1
2,41710,181955,200028,221906,2133-10-29 18:19:00,2133-10-29 18:28:00,0.075582,0.100,None,1
3,41710,181955,200028,221906,2133-10-29 18:28:00,2133-10-29 18:41:00,0.152844,0.140,None,1
4,41710,181955,200028,221906,2133-10-29 18:41:00,2133-10-29 18:49:00,0.107518,0.160,None,1
...,...,...,...,...,...,...,...,...,...,...
61349,42728,142693,299979,221749,2127-12-05 22:11:00,2127-12-05 22:42:00,1.606187,0.270,None,1
61350,42728,142693,299979,221749,2127-12-05 22:42:00,2127-12-06 00:42:00,4.142000,0.180,None,1
61351,42728,142693,299979,221749,2127-12-06 00:42:00,2127-12-06 01:22:00,1.381666,0.180,None,1
61352,42728,142693,299979,221749,2127-12-06 01:22:00,2127-12-06 03:20:00,3.056545,0.135,None,1


In [18]:
vaso_mv.to_hdf('data_icu.h5', key='vaso_mv', format='table') 

In [19]:
## carevure
## from https://gitlab.doc.ic.ac.uk/AIClinician/AIClinician/-/blob/master/AIClinician_Data_extract_MIMIC3_140219.ipynb

query = """
select inputevents_cv.subject_id, inputevents_cv.hadm_id, inputevents_cv.icustay_id,  inputevents_cv.itemid, inputevents_cv.charttime, inputevents_cv.amount, inputevents_cv.rate, -- rate, -- rateuom,

case when itemid in (30120,221906,30047) and rateuom='mcgkgmin' then round(cast(rate as numeric),3) -- norad
when itemid in (30120,221906,30047) and rateuom='mcgmin' then round(cast(rate/80 as numeric),3)  -- norad
when itemid in (30119,221289) and rateuom='mcgkgmin' then round(cast(rate as numeric),3) -- epi
when itemid in (30119,221289) and rateuom='mcgmin' then round(cast(rate/80 as numeric),3) -- epi
when itemid in (30051,222315) and rate > 0.2 then round(cast(rate*5/60  as numeric),3) -- vasopressin, in U/h
when itemid in (30051,222315) and rateuom='Umin' and rate < 0.2 then round(cast(rate*5  as numeric),3) -- vasopressin
when itemid in (30051,222315) and rateuom='Uhr' then round(cast(rate*5/60  as numeric),3) -- vasopressin
when itemid in (30128,221749,30127) and rateuom='mcgkgmin' then round(cast(rate*0.45  as numeric),3) -- phenyl
when itemid in (30128,221749,30127) and rateuom='mcgmin' then round(cast(rate*0.45 / 80  as numeric),3) -- phenyl
when itemid in (221662,30043,30307) and rateuom='mcgkgmin' then round(cast(rate*0.01   as numeric),3) -- dopa
when itemid in (221662,30043,30307) and rateuom='mcgmin' then round(cast(rate*0.01/80  as numeric),3) else null end as rate_std-- dopa
, case when inputevents_cv.subject_id in %(subject_ids)s then 1 else 0 end as has_vasopressors-- binary indicator of whether patients received vassos


-- case when rateuom='mcgkgmin' then 1 when rateuom='mcgmin' then 2 end as uom
from inputevents_cv
INNER JOIN icustays  ON icustays.icustay_id = inputevents_cv.icustay_id


where itemid in (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307) and rate is not null
and inputevents_cv.icustay_id in %(icu_ids)s

and inputevents_cv.charttime between icustays.intime and icustays.outtime -- in between the icu stay


order by inputevents_cv.icustay_id, inputevents_cv.itemid, inputevents_cv.charttime

"""
vaso_cv = pd.read_sql_query(query,connection, params={'subject_ids': tuple(subject_id_set), 'icu_ids': tuple(icustay_id_set)})

In [20]:
len(set(vaso_cv.subject_id))

6480

In [21]:
vaso_cv.to_hdf('data_icu.h5', key='vaso_cv') 


c:\Users\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\.venv\lib\site-packages\pandas\core\generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['amount'], dtype='object')]

  encoding=encoding,


## Mechanical Ventilation

In [22]:
# Identify The presence of a mechanical ventilation using settings
# from https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iii/concepts/durations/ventilation_classification.sql
query = """
select
  ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime
  -- case statement determining whether it is an instance of mech vent
  , max(
    case
      when itemid is null or value is null then 0 -- can't have null values
      when itemid = 720 and value != 'Other/Remarks' THEN 1  -- VentTypeRecorded
      when itemid = 223848 and value != 'Other' THEN 1
      when itemid = 223849 then 1 -- ventilator mode
      when itemid = 467 and value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
      when itemid in
        (
        445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
        , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
        , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
        , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
        , 543 -- PlateauPressure
        , 5865,5866,224707,224709,224705,224706 -- APRV pressure
        , 60,437,505,506,686,220339,224700 -- PEEP
        , 3459 -- high pressure relief
        , 501,502,503,224702 -- PCV
        , 223,667,668,669,670,671,672 -- TCPCV
        , 224701 -- PSVlevel
        )
        THEN 1
      else 0
    end
    ) as MechVent
    , max(
      case
        -- initiation of oxygen therapy indicates the ventilation has ended
        when itemid = 226732 and value in
        (
          'Nasal cannula', -- 153714 observations
          'Face tent', -- 24601 observations
          'Aerosol-cool', -- 24560 observations
          'Trach mask ', -- 16435 observations
          'High flow neb', -- 10785 observations
          'Non-rebreather', -- 5182 observations
          'Venti mask ', -- 1947 observations
          'Medium conc mask ', -- 1888 observations
          'T-piece', -- 1135 observations
          'High flow nasal cannula', -- 925 observations
          'Ultrasonic neb', -- 9 observations
          'Vapomist' -- 3 observations
        ) then 1
        when itemid = 467 and value in
        (
          'Cannula', -- 278252 observations
          'Nasal Cannula', -- 248299 observations
          -- 'None', -- 95498 observations
          'Face Tent', -- 35766 observations
          'Aerosol-Cool', -- 33919 observations
          'Trach Mask', -- 32655 observations
          'Hi Flow Neb', -- 14070 observations
          'Non-Rebreather', -- 10856 observations
          'Venti Mask', -- 4279 observations
          'Medium Conc Mask', -- 2114 observations
          'Vapotherm', -- 1655 observations
          'T-Piece', -- 779 observations
          'Hood', -- 670 observations
          'Hut', -- 150 observations
          'TranstrachealCat', -- 78 observations
          'Heated Neb', -- 37 observations
          'Ultrasonic Neb' -- 2 observations
        ) then 1
      else 0
      end
    ) as OxygenTherapy
    , max(
      case when itemid is null or value is null then 0
        -- extubated indicates ventilation event has ended
        when itemid = 640 and value = 'Extubated' then 1
        when itemid = 640 and value = 'Self Extubation' then 1
      else 0
      end
      )
      as Extubated
    , max(
      case when itemid is null or value is null then 0
        when itemid = 640 and value = 'Self Extubation' then 1
      else 0
      end
      )
      as SelfExtubated
from chartevents ce
INNER JOIN icustays  ON icustays.icustay_id = ce.icustay_id
where ce.value is not null
and ce.icustay_id in %(icu_ids)s
and ce.charttime between icustays.intime and icustays.outtime -- in between the icu stay

-- exclude rows marked as error
and (ce.error != 1 or ce.error IS NULL)
and itemid in
(
    -- the below are settings used to indicate ventilation
      720, 223849 -- vent mode
    , 223848 -- vent type
    , 445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
    , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
    , 218,436,535,444,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean ("RespPressure")
    , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
    , 543 -- PlateauPressure
    , 5865,5866,224707,224709,224705,224706 -- APRV pressure
    , 60,437,505,506,686,220339,224700 -- PEEP
    , 3459 -- high pressure relief
    , 501,502,503,224702 -- PCV
    , 223,667,668,669,670,671,672 -- TCPCV
    , 224701 -- PSVlevel

    -- the below are settings used to indicate extubation
    , 640 -- extubated

    -- the below indicate oxygen/NIV, i.e. the end of a mechanical vent event
    , 468 -- O2 Delivery Device#2
    , 469 -- O2 Delivery Mode
    , 470 -- O2 Flow (lpm)
    , 471 -- O2 Flow (lpm) #2
    , 227287 -- O2 Flow (additional cannula)
    , 226732 -- O2 Delivery Device(s)
    , 223834 -- O2 Flow

    -- used in both oxygen + vent calculation
    , 467 -- O2 Delivery Device
)
group by ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime
UNION DISTINCT
-- add in the extubation flags from procedureevents_mv
-- note that we only need the start time for the extubation
-- (extubation is always charted as ending 1 minute after it started)
select
  procedureevents_mv.subject_id, procedureevents_mv.hadm_id, procedureevents_mv.icustay_id, starttime as charttime
  , 0 as MechVent
  , 0 as OxygenTherapy
  , 1 as Extubated
  , case when itemid = 225468 then 1 else 0 end as SelfExtubated
from procedureevents_mv
INNER JOIN icustays  ON icustays.icustay_id = procedureevents_mv.icustay_id
where itemid in
(
  227194 -- "Extubation"
, 225468 -- "Unplanned Extubation (patient-initiated)"
, 225477 -- "Unplanned Extubation (non-patient initiated)"
)
and procedureevents_mv.icustay_id in %(icu_ids)s
and procedureevents_mv.starttime between icustays.intime and icustays.outtime -- in between the icu stay
;"""

vt = pd.read_sql_query(query, connection, params={'icu_ids': tuple(icustay_id_set)})

In [23]:
vt.head()

,subject_id,hadm_id,icustay_id,charttime,mechvent,oxygentherapy,extubated,selfextubated
0,3,145834,211552,2101-10-20 19:15:00,0,0,0,0
1,3,145834,211552,2101-10-20 20:00:00,1,0,0,0
2,3,145834,211552,2101-10-20 21:15:00,0,0,0,0
3,3,145834,211552,2101-10-20 23:15:00,1,0,0,0
4,3,145834,211552,2101-10-21 04:15:00,1,0,0,0


In [24]:
vt.to_hdf('data_icu.h5', key='mech_vent') 

In [25]:
## from https://gitlab.doc.ic.ac.uk/AIClinician/AIClinician/-/blob/master/AIClinician_Data_extract_MIMIC3_140219.ipynb

query = """

select
    ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime    -- case statement determining whether it is an instance of mech vent
    , max(
      case
        when itemid is null or value is null then 0 -- can't have null values
        when itemid = 720 and value != 'Other/Remarks' THEN 1  -- VentTypeRecorded
        when itemid = 467 and value = 'Ventilator' THEN 1 -- O2 delivery device == ventilator
        when itemid in
          (
          445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
          , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
          , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
          , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
          , 543 -- PlateauPressure
          , 5865,5866,224707,224709,224705,224706 -- APRV pressure
          , 60,437,505,506,686,220339,224700 -- PEEP
          , 3459 -- high pressure relief
          , 501,502,503,224702 -- PCV
          , 223,667,668,669,670,671,672 -- TCPCV
          , 157,158,1852,3398,3399,3400,3401,3402,3403,3404,8382,227809,227810 -- ETT
          , 224701 -- PSVlevel
          )
          THEN 1
        else 0
      end
      ) as MechVent
      , max(
        case when itemid is null or value is null then 0
          when itemid = 640 and value = 'Extubated' then 1
          when itemid = 640 and value = 'Self Extubation' then 1
        else 0
        end
        )
        as Extubated
      , max(
        case when itemid is null or value is null then 0
          when itemid = 640 and value = 'Self Extubation' then 1
        else 0
        end
        )
        as SelfExtubated


  from chartevents ce
  INNER JOIN icustays  ON icustays.icustay_id = ce.icustay_id
  where ce.value is not null
  and ce.icustay_id in %(icu_ids)s
  and ce.charttime between icustays.intime and icustays.outtime -- in between the icu stay
  and ce.itemid in
  (
      640 -- extubated
      , 720 -- vent type
      , 467 -- O2 delivery device
      , 445, 448, 449, 450, 1340, 1486, 1600, 224687 -- minute volume
      , 639, 654, 681, 682, 683, 684,224685,224684,224686 -- tidal volume
      , 218,436,535,444,459,224697,224695,224696,224746,224747 -- High/Low/Peak/Mean/Neg insp force ("RespPressure")
      , 221,1,1211,1655,2000,226873,224738,224419,224750,227187 -- Insp pressure
      , 543 -- PlateauPressure
      , 5865,5866,224707,224709,224705,224706 -- APRV pressure
      , 60,437,505,506,686,220339,224700 -- PEEP
      , 3459 -- high pressure relief
      , 501,502,503,224702 -- PCV
      , 223,667,668,669,670,671,672 -- TCPCV
      , 157,158,1852,3398,3399,3400,3401,3402,3403,3404,8382,227809,227810 -- ETT
      , 224701 -- PSVlevel
  )
  group by ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime


"""
mech_vent = pd.read_sql_query(query,connection, params={'icu_ids': tuple(icustay_id_set)})

In [26]:
mech_vent.head()

,subject_id,hadm_id,icustay_id,charttime,mechvent,extubated,selfextubated
0,3,145834,211552,2101-10-20 19:15:00,0,0,0
1,3,145834,211552,2101-10-20 20:00:00,1,0,0
2,3,145834,211552,2101-10-20 21:15:00,0,0,0
3,3,145834,211552,2101-10-20 23:15:00,1,0,0
4,3,145834,211552,2101-10-21 02:00:00,1,0,0


## Crystalloid and colloid bolus fluids

In [27]:
## colloid
## from https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iii/concepts/fluid_balance/crystalloid_bolus.sql
## Fluid bolus therapy is widely administered to patients with undifferentiated hypotension and for patients with severe sepsis
query = """

-- received colloid before admission
-- 226365  --  OR Colloid Intake
-- 226376  --  PACU Colloid Intake

with t1 as
(
  select
    mv.subject_id
  , mv.hadm_id  
  , mv.icustay_id
  , mv.starttime as charttime
  -- standardize the units to millilitres
  -- also metavision has floating point precision.. but we only care down to the mL
  , round(case
      when mv.amountuom = 'L'
        then mv.amount * 1000.0
      when mv.amountuom = 'ml'
        then mv.amount
    else null end) as amount
  from inputevents_mv mv
  INNER JOIN icustays ON icustays.icustay_id = mv.icustay_id
  where mv.itemid in
  (
    220864, --	Albumin 5%	7466 132 7466
    220862, --	Albumin 25%	9851 174 9851
    225174, --	Hetastarch (Hespan) 6%	82 1 82
    225795, --	Dextran 40	38 3 38
    225796  --  Dextran 70
    -- below ITEMIDs not in use
   -- 220861 | Albumin (Human) 20%
   -- 220863 | Albumin (Human) 4%
  )
  and mv.statusdescription != 'Rewritten'
  and mv.starttime between intime and outtime -- in between the icu stay
  and
  -- in MetaVision, these ITEMIDs never appear with a null rate
  -- so it is sufficient to check the rate is > 100
    (
      (mv.rateuom = 'mL/hour' and mv.rate > 100)
      OR (mv.rateuom = 'mL/min' and mv.rate > (100/60.0))
      OR (mv.rateuom = 'mL/kg/hour' and (mv.rate*mv.patientweight) > 100)
    )
)
, t2 as
(
  select
    cv.subject_id
  , cv.hadm_id  
  , cv.icustay_id
  , cv.charttime
  -- carevue always has units in millilitres (or null)
  , round(cv.amount) as amount
  from inputevents_cv cv
  INNER JOIN icustays  ON icustays.icustay_id = cv.icustay_id

  where cv.itemid in
  (
   30008 --	Albumin 5%
  ,30009 --	Albumin 25%
  ,42832 --	albumin 12.5%
  ,40548 --	ALBUMIN
  ,45403 --	albumin
  ,44203 --	Albumin 12.5%
  ,30181 -- Serum Albumin 5%
  ,46564 -- Albumin
  ,43237 -- 25% Albumin
  ,43353 -- Albumin (human) 25%

  ,30012 --	Hespan
  ,46313 --	6% Hespan

  ,30011 -- Dextran 40
  ,30016 -- Dextrose 10%
  ,42975 --	DEXTRAN DRIP
  ,42944 --	dextran
  ,46336 --	10% Dextran 40/D5W
  ,46729 --	Dextran
  ,40033 --	DEXTRAN
  ,45410 --	10% Dextran 40
  ,42731 -- Dextran40 10%
  )
  and cv.amount > 100
  and cv.amount < 2000
  and cv.charttime between intime and outtime -- in between the icu stay

)

-- some colloids are charted in chartevents
, t3 as
(
  select
    ce.subject_id
  , ce.hadm_id 
  , ce.icustay_id
  , ce.charttime
  -- carevue always has units in millilitres (or null)
  , round(ce.valuenum) as amount
  from chartevents ce
  INNER JOIN icustays  ON icustays.icustay_id = ce.icustay_id
  where ce.itemid in
  (
      2510 --	DEXTRAN LML 10%
    , 3087 --	DEXTRAN 40  10%
    , 6937 --	Dextran
    , 3087 -- DEXTRAN 40  10%
    , 3088 --	DEXTRAN 40%
  )
  and ce.valuenum is not null
  and ce.valuenum > 100
  and ce.valuenum < 2000
  and ce.charttime between intime and outtime -- in between the icu stay

)

select
    subject_id
  ,  hadm_id
  ,  icustay_id
  , charttime
  , sum(amount) as colloid_bolus
from t1
-- just because the rate was high enough, does *not* mean the final amount was
where amount > 100
--and icustay_id in %(icu_ids)s
group by t1.subject_id, t1.hadm_id, t1.icustay_id, t1.charttime
UNION ALL
select
    subject_id
  ,  hadm_id
  ,  icustay_id
  , charttime
  , sum(amount) as colloid_bolus
from t2
--where icustay_id in %(icu_ids)s
group by t2.subject_id, t2.hadm_id, t2.icustay_id, t2.charttime
UNION ALL 
select
    subject_id
  ,  hadm_id
  ,  icustay_id
  , charttime
  , sum(amount) as colloid_bolus
from t3
--where icustay_id in %(icu_ids)s
group by t3.subject_id, t3.hadm_id, t3.icustay_id, t3.charttime
order by subject_id, hadm_id, icustay_id, charttime;
"""
colloid_bolus= pd.read_sql(query, connection)
#, params={'icu_ids': tuple(icustay_id_set)}

In [28]:
colloid_bolus

,subject_id,hadm_id,icustay_id,charttime,colloid_bolus
0,20,157681.0,264490,2183-04-28 23:30:00,500.0
1,36,122659.0,211200,2131-05-18 23:00:00,500.0
2,114,178393.0,258626,2146-08-30 05:00:00,500.0
3,114,178393.0,258626,2146-08-30 12:00:00,500.0
4,115,114585.0,232514,2194-10-17 16:00:00,500.0
...,...,...,...,...,...
12174,99982,151454.0,221194,2156-11-29 21:35:00,500.0
12175,99982,151454.0,221194,2156-11-29 22:16:00,500.0
12176,99982,151454.0,221194,2156-11-29 22:56:00,500.0
12177,99982,151454.0,221194,2156-12-04 11:26:00,250.0


In [29]:
# apply mask to receive patients with the queried disease and the specific hadm_id (visit in icu) that the disease was registered. 
colloid_bolus_f = colloid_bolus[colloid_bolus.icustay_id.isin(list(icustay_id_set))]

In [30]:
colloid_bolus_f

,subject_id,hadm_id,icustay_id,charttime,colloid_bolus
0,20,157681.0,264490,2183-04-28 23:30:00,500.0
2,114,178393.0,258626,2146-08-30 05:00:00,500.0
3,114,178393.0,258626,2146-08-30 12:00:00,500.0
4,115,114585.0,232514,2194-10-17 16:00:00,500.0
5,115,114585.0,232514,2194-10-17 22:00:00,500.0
...,...,...,...,...,...
12174,99982,151454.0,221194,2156-11-29 21:35:00,500.0
12175,99982,151454.0,221194,2156-11-29 22:16:00,500.0
12176,99982,151454.0,221194,2156-11-29 22:56:00,500.0
12177,99982,151454.0,221194,2156-12-04 11:26:00,250.0


In [31]:
colloid_bolus_f[colloid_bolus_f.icustay_id == 226014]

,subject_id,hadm_id,icustay_id,charttime,colloid_bolus
89,728,174088.0,226014,2154-03-15 22:00:00,250.0
90,728,174088.0,226014,2154-03-15 23:00:00,250.0


In [32]:
colloid_bolus_f.to_hdf('data_icu.h5', key='colloid_bolus') 

In [33]:
## crystalloid
## Crystalloid fluids are a subset of intravenous solutions that are frequently used in the clinical setting. 
# Crystalloid fluids are the first choice for fluid resuscitation in the presence of hypovolemia, hemorrhage, 
# sepsis, and dehydration

query = """
with t1 as
(
  select
    mv.subject_id
  ,  mv.hadm_id
  ,  mv.icustay_id
  , mv.starttime as charttime
  -- standardize the units to millilitres
  -- also metavision has floating point precision.. but we only care down to the mL
  , round(case
      when mv.amountuom = 'L'
        then mv.amount * 1000.0
      when mv.amountuom = 'ml'
        then mv.amount
    else null end) as amount
  from inputevents_mv mv
  INNER JOIN icustays ON icustays.icustay_id = mv.icustay_id
  where mv.itemid in
  (
    -- 225943 Solution
    225158, -- NaCl 0.9%
    225828, -- LR
    225944, -- Sterile Water
    225797, -- Free Water
	  225159, -- NaCl 0.45%
	  -- 225161, -- NaCl 3% (Hypertonic Saline)
	  225823, -- D5 1/2NS
	  225825, -- D5NS
	  225827, -- D5LR
	  225941, -- D5 1/4NS
	  226089 -- Piggyback
  )
  and mv.statusdescription != 'Rewritten'
  and mv.starttime between intime and outtime -- in between the icu stay
  and
  -- in MetaVision, these ITEMIDs appear with a null rate IFF endtime=starttime + 1 minute
  -- so it is sufficient to:
  --    (1) check the rate is > 240 if it exists or
  --    (2) ensure the rate is null and amount > 240 ml
    (
      (mv.rate is not null and mv.rateuom = 'mL/hour' and mv.rate > 248)
      OR (mv.rate is not null and mv.rateuom = 'mL/min' and mv.rate > (248/60.0))
      OR (mv.rate is null and mv.amountuom = 'L' and mv.amount > 0.248)
      OR (mv.rate is null and mv.amountuom = 'ml' and mv.amount > 248)
    )
)
, t2 as
(
  select
    cv.subject_id
  , cv.hadm_id  
  , cv.icustay_id
  , cv.charttime
  -- carevue always has units in millilitres
  , round(cv.amount) as amount
  from inputevents_cv cv
  INNER JOIN icustays  ON icustays.icustay_id = cv.icustay_id
  where cv.itemid in
  (
    30015 -- "D5/.45NS" -- mixed colloids and crystalloids
  , 30018 --	.9% Normal Saline
  , 30020 -- .45% Normal Saline
  , 30021 --	Lactated Ringers
  , 30058 --	Free Water Bolus
  , 30060 -- D5NS
  , 30061 -- D5RL
  , 30063 --	IV Piggyback
  , 30065 --	Sterile Water
  -- , 30143 -- 3% Normal Saline
  , 30159 -- D5 Ringers Lact.
  , 30160 -- D5 Normal Saline
  , 30169 --	Sterile H20_GU
  , 30190 -- NS .9%
  , 40850 --	ns bolus
  , 41491 --	fluid bolus
  , 42639 --	bolus
  , 42187 --	free h20
  , 43819 --	1:1 NS Repletion.
  , 41430 --	free water boluses
  , 40712 --	free H20
  , 44160 --	BOLUS
  , 42383 --	cc for cc replace
  , 42297 --	Fluid bolus
  , 42453 --	Fluid Bolus
  , 40872 --	free water
  , 41915 --	FREE WATER
  , 41490 --	NS bolus
  , 46501 --	H2O Bolus
  , 45045 --	WaterBolus
  , 41984 --	FREE H20
  , 41371 --	ns fluid bolus
  , 41582 --	free h20 bolus
  , 41322 --	rl bolus
  , 40778 --	Free H2O
  , 41896 --	ivf boluses
  , 41428 --	ns .9% bolus
  , 43936 --	FREE WATER BOLUSES
  , 44200 --	FLUID BOLUS
  , 41619 --	frfee water boluses
  , 40424 --	free H2O
  , 41457 --	Free H20 intake
  , 41581 --	Water bolus
  , 42844 --	NS fluid bolus
  , 42429 --	Free water
  , 41356 --	IV Bolus
  , 40532 --	FREE H2O
  , 42548 --	NS Bolus
  , 44184 --	LR Bolus
  , 44521 --	LR bolus
  , 44741 --	NS FLUID BOLUS
  , 44126 --	fl bolus
  , 44110 --	RL BOLUS
  , 44633 --	ns boluses
  , 44983 --	Bolus NS
  , 44815 --	LR BOLUS
  , 43986 --	iv bolus
  , 45079 --	500 cc ns bolus
  , 46781 --	lr bolus
  , 45155 --	ns cc/cc replacement
  , 43909 --	H20 BOlus
  , 41467 --	NS IV bolus
  , 44367 --	LR
  , 41743 --	water bolus
  , 40423 --	Bolus
  , 44263 --	fluid bolus ns
  , 42749 --	fluid bolus NS
  , 45480 --	500cc ns bolus
  , 44491 --	.9NS bolus
  , 41695 --	NS fluid boluses
  , 46169 --	free water bolus.
  , 41580 --	free h2o bolus
  , 41392 --	ns b
  , 45989 --	NS Fluid Bolus
  , 45137 --	NS cc/cc
  , 45154 --	Free H20 bolus
  , 44053 --	normal saline bolus
  , 41416 --	free h2o boluses
  , 44761 --	Free H20
  , 41237 --	ns fluid boluses
  , 44426 --	bolus ns
  , 43975 --	FREE H20 BOLUSES
  , 44894 --	N/s 500 ml bolus
  , 41380 --	nsbolus
  , 42671 --	free h2o
  )
  and cv.amount > 248
  and cv.amount <= 2000
  and cv.amountuom = 'ml'
  and cv.charttime between intime and outtime -- in between the icu stay

)
select
    subject_id
  , hadm_id 
  , icustay_id
  , charttime
  , sum(amount) as crystalloid_bolus
from t1
-- just because the rate was high enough, does *not* mean the final amount was
where amount > 248
group by t1.subject_id, t1.hadm_id, t1.icustay_id, t1.charttime
UNION
select
    subject_id
  , hadm_id 
  ,  icustay_id
  , charttime
  , sum(amount) as crystalloid_bolus
from t2
group by t2.subject_id, t2.hadm_id, t2.icustay_id, t2.charttime
order by subject_id, hadm_id, icustay_id, charttime;

"""
crystalloid_bolus= pd.read_sql(query, connection)




In [34]:
# apply mask to receive patients with the queried disease and the specific hadm_id (visit in icu) that the disease was registered. 
crystalloid_bolus_f = crystalloid_bolus[crystalloid_bolus.icustay_id.isin(list(icustay_id_set))]

In [35]:
crystalloid_bolus_f

,subject_id,hadm_id,icustay_id,charttime,crystalloid_bolus
0,3,145834.0,211552,2101-10-21 11:00:00,500.0
1,3,145834.0,211552,2101-10-21 14:00:00,820.0
2,3,145834.0,211552,2101-10-21 17:30:00,500.0
3,3,145834.0,211552,2101-10-21 20:00:00,500.0
4,3,145834.0,211552,2101-10-21 22:30:00,500.0
...,...,...,...,...,...
155070,99982,151454.0,221194,2156-11-29 12:59:00,1000.0
155071,99982,151454.0,221194,2156-11-29 13:22:00,1000.0
155072,99982,151454.0,221194,2156-11-29 15:57:00,250.0
155073,99982,151454.0,221194,2156-11-29 16:25:00,1000.0


In [36]:
crystalloid_bolus_f.to_hdf('data_icu.h5', key='crystalloid_bolus') 

## Antiobiotics

In [37]:
query = """

with t1 as
(
  select
    subject_id, hadm_id, icustay_id, startdate, enddate, drug, drug_name_generic
    , route
    , case
      when lower(drug) like '%adoxa%' then 1
      when lower(drug) like '%ala-tet%' then 1
      when lower(drug) like '%alodox%' then 1
      when lower(drug) like '%amikacin%' then 1
      when lower(drug) like '%amikin%' then 1
      when lower(drug) like '%amoxicillin%' then 1
      when lower(drug) like '%amoxicillin%clavulanate%' then 1
      when lower(drug) like '%clavulanate%' then 1
      when lower(drug) like '%ampicillin%' then 1
      when lower(drug) like '%augmentin%' then 1
      when lower(drug) like '%avelox%' then 1
      when lower(drug) like '%avidoxy%' then 1
      when lower(drug) like '%azactam%' then 1
      when lower(drug) like '%azithromycin%' then 1
      when lower(drug) like '%aztreonam%' then 1
      when lower(drug) like '%axetil%' then 1
      when lower(drug) like '%bactocill%' then 1
      when lower(drug) like '%bactrim%' then 1
      when lower(drug) like '%bethkis%' then 1
      when lower(drug) like '%biaxin%' then 1
      when lower(drug) like '%bicillin l-a%' then 1
      when lower(drug) like '%cayston%' then 1
      when lower(drug) like '%cefazolin%' then 1
      when lower(drug) like '%cedax%' then 1
      when lower(drug) like '%cefoxitin%' then 1
      when lower(drug) like '%ceftazidime%' then 1
      when lower(drug) like '%cefaclor%' then 1
      when lower(drug) like '%cefadroxil%' then 1
      when lower(drug) like '%cefdinir%' then 1
      when lower(drug) like '%cefditoren%' then 1
      when lower(drug) like '%cefepime%' then 1
      when lower(drug) like '%cefotetan%' then 1
      when lower(drug) like '%cefotaxime%' then 1
      when lower(drug) like '%cefpodoxime%' then 1
      when lower(drug) like '%cefprozil%' then 1
      when lower(drug) like '%ceftibuten%' then 1
      when lower(drug) like '%ceftin%' then 1
      when lower(drug) like '%cefuroxime %' then 1
      when lower(drug) like '%cefuroxime%' then 1
      when lower(drug) like '%cephalexin%' then 1
      when lower(drug) like '%chloramphenicol%' then 1
      when lower(drug) like '%cipro%' then 1
      when lower(drug) like '%ciprofloxacin%' then 1
      when lower(drug) like '%claforan%' then 1
      when lower(drug) like '%clarithromycin%' then 1
      when lower(drug) like '%cleocin%' then 1
      when lower(drug) like '%clindamycin%' then 1
      when lower(drug) like '%cubicin%' then 1
      when lower(drug) like '%dicloxacillin%' then 1
      when lower(drug) like '%doryx%' then 1
      when lower(drug) like '%doxycycline%' then 1
      when lower(drug) like '%duricef%' then 1
      when lower(drug) like '%dynacin%' then 1
      when lower(drug) like '%ery-tab%' then 1
      when lower(drug) like '%eryped%' then 1
      when lower(drug) like '%eryc%' then 1
      when lower(drug) like '%erythrocin%' then 1
      when lower(drug) like '%erythromycin%' then 1
      when lower(drug) like '%factive%' then 1
      when lower(drug) like '%flagyl%' then 1
      when lower(drug) like '%fortaz%' then 1
      when lower(drug) like '%furadantin%' then 1
      when lower(drug) like '%garamycin%' then 1
      when lower(drug) like '%gentamicin%' then 1
      when lower(drug) like '%kanamycin%' then 1
      when lower(drug) like '%keflex%' then 1
      when lower(drug) like '%ketek%' then 1
      when lower(drug) like '%levaquin%' then 1
      when lower(drug) like '%levofloxacin%' then 1
      when lower(drug) like '%lincocin%' then 1
      when lower(drug) like '%macrobid%' then 1
      when lower(drug) like '%macrodantin%' then 1
      when lower(drug) like '%maxipime%' then 1
      when lower(drug) like '%mefoxin%' then 1
      when lower(drug) like '%metronidazole%' then 1
      when lower(drug) like '%minocin%' then 1
      when lower(drug) like '%minocycline%' then 1
      when lower(drug) like '%monodox%' then 1
      when lower(drug) like '%monurol%' then 1
      when lower(drug) like '%morgidox%' then 1
      when lower(drug) like '%moxatag%' then 1
      when lower(drug) like '%moxifloxacin%' then 1
      when lower(drug) like '%myrac%' then 1
      when lower(drug) like '%nafcillin sodium%' then 1
      when lower(drug) like '%nicazel doxy 30%' then 1
      when lower(drug) like '%nitrofurantoin%' then 1
      when lower(drug) like '%noroxin%' then 1
      when lower(drug) like '%ocudox%' then 1
      when lower(drug) like '%ofloxacin%' then 1
      when lower(drug) like '%omnicef%' then 1
      when lower(drug) like '%oracea%' then 1
      when lower(drug) like '%oraxyl%' then 1
      when lower(drug) like '%oxacillin%' then 1
      when lower(drug) like '%pc pen vk%' then 1
      when lower(drug) like '%pce dispertab%' then 1
      when lower(drug) like '%panixine%' then 1
      when lower(drug) like '%pediazole%' then 1
      when lower(drug) like '%penicillin%' then 1
      when lower(drug) like '%periostat%' then 1
      when lower(drug) like '%pfizerpen%' then 1
      when lower(drug) like '%piperacillin%' then 1
      when lower(drug) like '%tazobactam%' then 1
      when lower(drug) like '%primsol%' then 1
      when lower(drug) like '%proquin%' then 1
      when lower(drug) like '%raniclor%' then 1
      when lower(drug) like '%rifadin%' then 1
      when lower(drug) like '%rifampin%' then 1
      when lower(drug) like '%rocephin%' then 1
      when lower(drug) like '%smz-tmp%' then 1
      when lower(drug) like '%septra%' then 1
      when lower(drug) like '%septra ds%' then 1
      when lower(drug) like '%septra%' then 1
      when lower(drug) like '%solodyn%' then 1
      when lower(drug) like '%spectracef%' then 1
      when lower(drug) like '%streptomycin sulfate%' then 1
      when lower(drug) like '%sulfadiazine%' then 1
      when lower(drug) like '%sulfamethoxazole%' then 1
      when lower(drug) like '%trimethoprim%' then 1
      when lower(drug) like '%sulfatrim%' then 1
      when lower(drug) like '%sulfisoxazole%' then 1
      when lower(drug) like '%suprax%' then 1
      when lower(drug) like '%synercid%' then 1
      when lower(drug) like '%tazicef%' then 1
      when lower(drug) like '%tetracycline%' then 1
      when lower(drug) like '%timentin%' then 1
      when lower(drug) like '%tobi%' then 1
      when lower(drug) like '%tobramycin%' then 1
      when lower(drug) like '%trimethoprim%' then 1
      when lower(drug) like '%unasyn%' then 1
      when lower(drug) like '%vancocin%' then 1
      when lower(drug) like '%vancomycin%' then 1
      when lower(drug) like '%vantin%' then 1
      when lower(drug) like '%vibativ%' then 1
      when lower(drug) like '%vibra-tabs%' then 1
      when lower(drug) like '%vibramycin%' then 1
      when lower(drug) like '%zinacef%' then 1
      when lower(drug) like '%zithromax%' then 1
      when lower(drug) like '%zmax%' then 1
      when lower(drug) like '%zosyn%' then 1
      when lower(drug) like '%zyvox%' then 1
    else 0
    end as antibiotic
  from prescriptions
  where drug_type in ('MAIN','ADDITIVE')
  -- we exclude routes via the eye, ears, or topically
  and route not in ('OU','OS','OD','AU','AS','AD', 'TP')
  and lower(route) not like '%ear%'
  and lower(route) not like '%eye%'
  -- we exclude certain types of antibiotics: topical creams, gels, desens, etc
  and lower(drug) not like '%cream%'
  and lower(drug) not like '%desensitization%'
  and lower(drug) not like '%ophth oint%'
  and lower(drug) not like '%gel%'
  -- other routes not sure about...
  -- for sure keep: ('IV','PO','PO/NG','ORAL', 'IV DRIP', 'IV BOLUS')
  -- ? VT, PB, PR, PL, NS, NG, NEB, NAS, LOCK, J TUBE, IVT
  -- ? IT, IRR, IP, IO, INHALATION, IN, IM
  -- ? IJ, IH, G TUBE, DIALYS
  -- ?? enemas??
)
select t1.subject_id, t1.hadm_id, t1.icustay_id, t1.startdate, t1.enddate, t1.drug, t1.drug_name_generic
    , t1.route, t1.antibiotic
from t1
INNER JOIN icustays  ON icustays.icustay_id = t1.icustay_id
where antibiotic = 1
and startdate between intime and outtime -- in between the icu stay

--group by drug --, drug_name_generic
--order by numobs desc;


"""

antiobiotics= pd.read_sql(query, connection)

In [38]:
# apply mask to receive patients with the queried disease and the specific hadm_id (visit in icu) that the disease was registered. 
antiobiotics_f = antiobiotics[antiobiotics.icustay_id.isin(list(icustay_id_set))]
antiobiotics_f

,subject_id,hadm_id,icustay_id,startdate,enddate,drug,drug_name_generic,route,antibiotic
0,13,143045,263738,2167-01-09,2167-01-11,Vancomycin HCl,None,IV,1
1,9,150750,220597,2149-11-10,2149-11-11,Levofloxacin,Levofloxacin,NG,1
2,4,185777,294638,2191-03-17,2191-03-18,Vancomycin HCl,None,IV,1
3,12,112213,232669,2104-08-11,2104-08-11,Metronidazole,None,IV,1
4,12,112213,232669,2104-08-11,2104-08-12,Levofloxacin,None,IV,1
...,...,...,...,...,...,...,...,...,...
100923,99957,148574,257127,2157-02-21,2157-02-21,Rifampin,Rifampin,PO,1
100924,99957,148574,257127,2157-02-21,2157-02-21,Piperacillin-Tazobactam Na,None,IV,1
100925,99957,148574,257127,2157-02-21,2157-02-21,Piperacillin-Tazobactam Na,None,IV,1
100926,99955,108494,243255,2139-07-18,2139-07-19,Vancomycin,None,IV,1


In [39]:
antiobiotics_f.to_hdf('data_icu.h5', key='antibiotics') 

c:\Users\Maria\Desktop\Work\Projects\MIMIC\Representation-Clustering\.venv\lib\site-packages\pandas\core\generic.py:2718: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['drug', 'drug_name_generic', 'route'], dtype='object')]

  encoding=encoding,


## Diagnoses

In [40]:
diagnoses = pd.read_sql("""

WITH filter_diagnoses AS 
(
    SELECT diagnoses_icd.subject_id, diagnoses_icd.hadm_id, diagnoses_icd.icd9_code, d_icd_diagnoses.short_title as icd9_title
    FROM diagnoses_icd 
    INNER JOIN d_icd_diagnoses 
        ON diagnoses_icd.icd9_code = d_icd_diagnoses.icd9_code
    WHERE diagnoses_icd.subject_id IN %(subject_id_set)s
    AND diagnoses_icd.hadm_id IN %(hadm_id_set)s
)


SELECT *
FROM filter_diagnoses
ORDER BY subject_id

""", con= connection, params={'subject_id_set': tuple(subject_id_set), 'hadm_id_set': tuple(hadm_id_set)})

In [41]:
diagnoses

,subject_id,hadm_id,icd9_code,icd9_title
0,3,145834,4254,Prim cardiomyopathy NEC
1,3,145834,2639,Protein-cal malnutr NOS
2,3,145834,6826,Cellulitis of leg
3,3,145834,4280,CHF NOS
4,3,145834,41071,"Subendo infarct, initial"
...,...,...,...,...
367634,99999,113369,4139,Angina pectoris NEC/NOS
367635,99999,113369,75612,Spondylolisthesis
367636,99999,113369,7861,Stridor
367637,99999,113369,4019,Hypertension NOS


In [42]:
diagnoses.to_hdf('data_icu.h5', key='diagnoses') 